In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

2022-10-10 10:25:47.434360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 10:25:47.535613: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-10 10:25:47.539076: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-10 10:25:47.539088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [4]:
x_train = train_datagen.flow_from_directory(r'dataset/train',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical')

x_test = test_datagen.flow_from_directory(r'dataset/test',
                                          target_size=(64, 64),
                                          batch_size=5,
                                          color_mode='grayscale',
                                          class_mode='categorical') 


Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [5]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [6]:
model=Sequential()

2022-10-10 10:25:49.489000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-10 10:25:49.489046: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-10 10:25:49.489074: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vaishnav-pt5653): /proc/driver/nvidia/version does not exist
2022-10-10 10:25:49.489468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


In [8]:
model.add(Flatten())

In [9]:
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=6,activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:

model.fit_generator(x_train,
                    steps_per_epoch = len(x_train) , 
                    epochs = 20, 
                    validation_data = x_test,
                    validation_steps = len(x_test) )

Epoch 1/20


/tmp/ipykernel_18088/3854099306.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,


119/119 [==============================] - 1s 5ms/step - loss: 1.5467 - accuracy: 0.3502 - val_loss: 0.7810 - val_accuracy: 0.7667
Epoch 2/20
119/119 [==============================] - 1s 5ms/step - loss: 0.7902 - accuracy: 0.6768 - val_loss: 0.6201 - val_accuracy: 0.7000
Epoch 3/20
119/119 [==============================] - 1s 5ms/step - loss: 0.4994 - accuracy: 0.8199 - val_loss: 0.4606 - val_accuracy: 0.8333
Epoch 4/20
119/119 [==============================] - 1s 5ms/step - loss: 0.3766 - accuracy: 0.8569 - val_loss: 0.3547 - val_accuracy: 0.9333
Epoch 5/20
119/119 [==============================] - 1s 5ms/step - loss: 0.3274 - accuracy: 0.8754 - val_loss: 0.2926 - val_accuracy: 0.9333
Epoch 6/20
119/119 [==============================] - 1s 5ms/step - loss: 0.2095 - accuracy: 0.9209 - val_loss: 0.2593 - val_accuracy: 0.9333
Epoch 7/20
119/119 [==============================] - 1s 5ms/step - loss: 0.2230 - accuracy: 0.9276 - val_loss: 0.1528 - val_accuracy: 0.9667
Epoch 8/20
119/11

In [13]:
model.save('gesture.h5')
model_json=model.to_json()
with open("model-bw.json","w") as json_file:json_file.write(model_json)